In [1]:
import torch
import torch.nn as nn
import matplotlib.pyplot as plt
%run ~/violin-renderer/src/misc/parse.ipynb
# %run ~/violin-renderer/src/misc/randomizer.ipynb

In [2]:
HOME_DIR = "/home/wanninglu"

In [3]:
# initialize GPU to move model/tensors onto
device = torch.device("cuda:2" if torch.cuda.is_available() else "cpu")

In [4]:
# load all the datasets
training_X, training_y, testing_X, testing_y = load_from_paths(HOME_DIR)

In [5]:
class MLP(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super().__init__()
        self.linear1 = nn.Linear(input_size, hidden_size)
        self.actfn = nn.ReLU()
        self.linear2 = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        x = self.linear1(x)
        x = self.actfn(x)
        x = self.linear2(x)
        return x

In [6]:
# initialize the MLP
model = MLP(3, 4, 2)

# transfer model to GPU
model.to(device)

MLP(
  (linear1): Linear(in_features=3, out_features=4, bias=True)
  (actfn): ReLU()
  (linear2): Linear(in_features=4, out_features=2, bias=True)
)

In [7]:
# Define our loss function (mean squared error) to be used in the grad descent step
loss = nn.MSELoss()

# Performs the gradient descent steps
optimizer = torch.optim.SGD(model.parameters(), lr=1e-6)

In [8]:
# Trains the model inputted into the function.
#
# @param model The model object to be trained
# @param optimizer The optimizing equation to use to train the model
# @param input_notes The training input data
# @param truth The actual output for the corresponding input
# @param loss_module Equation for calculating the difference between generated and actual output
# @param num_epochs Number of cycles to train the model
def train_model_loop(model, optimizer, input_notes, truth, loss_module, num_epochs=5000):
    # Set model to train mode
    model.train()

    # Training loop
    for epoch in range(num_epochs):

        ## Step 1: Move input data to device (only strictly necessary if we use GPU)
        input_notes = input_notes.to(device)
        truth = truth.to(device)

        ## Step 2: Run the model on the input data
        preds = model(input_notes)

        ## Step 3: Calculate the loss
        loss = loss_module(preds, truth)

        ## Step 4: Perform backpropagation
        # Before calculating the gradients, we need to ensure that they are all zero.
        # The gradients would not be overwritten, but actually added to the existing ones.
        optimizer.zero_grad()
        # Perform backpropagation
        loss.backward()

        ## Step 5: Update the parameters
        optimizer.step()

In [9]:
# train for EACH file (warning: may take a while!!)
def train_model1():
    for (input_notes, truth) in zip(training_X, training_y):
        new_input_notes = torch.Tensor(input_notes)
        new_truth = torch.Tensor(truth)

        train_model_loop(model, optimizer, new_input_notes, new_truth, loss)
    torch.save(model.state_dict(), 'mlp.pt')

In [15]:
# consolidates all the training data into one big matrix
def train_model2():
    all_input = []
    all_truth = []
    for (input_notes, truth) in zip(training_X, training_y):
        all_input.extend(input_notes)
        all_truth.extend(truth)
        
    all_input = torch.Tensor(all_input)
    all_truth = torch.Tensor(all_truth)
    train_model_loop(model, optimizer, all_input, all_truth, loss, 10000)
    torch.save(model.state_dict(), 'mlp2.pt')

In [10]:
# this creates a dictionary of outputs for all pieces in the testing dataset
#
# @returns a dictionary mapping with key of file path and value of csv values
def generate_output():
    
    testing_results = {}
    test_paths = []

    file = open('file_paths/testing_truth.txt','r')
    lines = file.readlines()
    for line in lines:
        test_paths.append(line.strip())
    
    # generating an output for each piece in the testing input dataset
    for i in range(len(testing_X)):
        new_test_input = torch.Tensor(testing_X[i])
        new_test_input = new_test_input.to(device)
        
        y_test = model(new_test_input)
        y_test = y_test.tolist()
        for j in range(len(y_test)):
            y_test[j].append(testing_X[i][j][2])

        testing_results[test_paths[i]] = y_test

    return testing_results

In [11]:
def MSE_error():
    testing_results = generate_output()
    loss_values = []
    for output_path, truth in zip(testing_results, testing_y):
        output = testing_results[output_path]

        print(output_path)

        # pitch was only added so the result is able to be synthesized, we can remove it here
        for i in range(len(output)):
            output[i].pop()
        output = torch.Tensor(output)
        truth = torch.Tensor(truth)
        loss_value = loss(output, truth) / len(output)
        loss_values.append(loss_value)

    return loss_values